# Assignment 3 — Task 2: Spatial Visualization

**Goal:** Create choropleth map showing property values across San Francisco ZIP codes

In [ ]:
import pandas as pd
import altair as alt
import geopandas as gpd
import json

alt.data_transformers.disable_max_rows()
print(f"Altair {alt.__version__}")

In [ ]:
# Load data
df = pd.read_parquet('sf_property_data_clean.parquet')
zipcodes_gdf = gpd.read_file('San_Francisco_ZIP_Codes_20251020.geojson')

# Perform spatial join to add ZIP codes
print("Performing spatial join...")
df_clean = df.dropna(subset=['latitude', 'longitude', 'year', 'total_assessed_value'])

# Create GeoDataFrame from properties
props_gdf = gpd.GeoDataFrame(
    df_clean,
    geometry=gpd.points_from_xy(df_clean['longitude'], df_clean['latitude']),
    crs="EPSG:4326"
)

# Ensure ZIP GeoDataFrame has correct CRS
if zipcodes_gdf.crs is None:
    zipcodes_gdf = zipcodes_gdf.set_crs("EPSG:4326")
elif zipcodes_gdf.crs.to_string() != "EPSG:4326":
    zipcodes_gdf = zipcodes_gdf.to_crs("EPSG:4326")

# Find ZIP code column
zip_field = None
for col in zipcodes_gdf.columns:
    if any(k in col.lower() for k in ['zip', 'postal', 'zcta']):
        zip_field = col
        break

if zip_field is None:
    raise ValueError("Could not find ZIP code column in GeoJSON")

print(f"Using ZIP field: {zip_field}")

# Spatial join
joined = gpd.sjoin(props_gdf, zipcodes_gdf[[zip_field, 'geometry']], how='left', predicate='within')
joined = joined.rename(columns={zip_field: 'zip_code'})
joined['zip_code'] = joined['zip_code'].astype(str)
joined = joined.dropna(subset=['zip_code'])

print(f"Spatial join complete: {len(joined):,} properties with ZIP codes")

# Aggregate by ZIP and year
zip_yearly = joined.groupby(['zip_code', 'year']).agg({
    'total_assessed_value': ['median', 'mean', 'count']
}).reset_index()
zip_yearly.columns = ['zip_code', 'year', 'median_value', 'mean_value', 'property_count']

print(f"ZIP-year combinations: {len(zip_yearly):,}")
print(f"Years: {zip_yearly['year'].min()} - {zip_yearly['year'].max()}")
print(f"Unique ZIPs: {zip_yearly['zip_code'].nunique()}")

## Spatial Question

**How do property values vary spatially across San Francisco ZIP codes, and how have these spatial patterns evolved over time (2015-2023)?**

## Design Progression

#### Alternative 1: Static Choropleth (2023 Only)

**Design:** ZIP code polygons colored by median property value for 2023 only.

**Advantages:**
- Clear spatial patterns immediately visible
- Easy-to-read color encoding with distinct boundaries
- No interaction complexity - straightforward interpretation
- ZIP boundaries prominently displayed

**Limitations:**
- Shows only one time point - cannot explore 9 years of data
- No way to see temporal trends or compare years
- Missing critical temporal dimension of the research question
- Static snapshot doesn't reveal COVID impact or recovery

**Why rejected:** Cannot address the temporal component of the spatial question. Missing critical evolution aspect.


In [ ]:
# Merge geometry with 2023 data
zipcodes_gdf['zip_code'] = zipcodes_gdf[zip_field].astype(str)
zip_2023 = zipcodes_gdf[['zip_code', 'geometry']].merge(
    zip_yearly[zip_yearly['year'] == 2023], 
    on='zip_code', 
    how='left'
)

# Calculate consistent scale across all years
p02 = zip_yearly['median_value'].quantile(0.02)
p98 = zip_yearly['median_value'].quantile(0.98)

print(f"Value range: ${p02:,.0f} - ${p98:,.0f}")

# Convert to GeoJSON format for Altair
zip_2023_json = json.loads(zip_2023.to_json())

alt.Chart(alt.Data(values=zip_2023_json['features'])).mark_geoshape(
    stroke='white', 
    strokeWidth=1
).encode(
    color=alt.Color(
        'properties.median_value:Q',
        scale=alt.Scale(scheme='viridis', domain=[p02, p98]),
        title='Median Value ($)'
    ),
    tooltip=[
        alt.Tooltip('properties.zip_code:N', title='ZIP'),
        alt.Tooltip('properties.median_value:Q', format='$,.0f', title='Median Value'),
        alt.Tooltip('properties.property_count:Q', format=',', title='Properties')
    ]
).project(
    type='mercator'
).properties(
    width=600, 
    height=500, 
    title='SF Property Values by ZIP Code (2023)'
)

#### Alternative 2: Dot Density Map

**Design:** Individual properties plotted as points with color-encoded values over ZIP boundaries.

**Advantages:**
- Shows individual property distribution within ZIPs
- Reveals density patterns and clusters
- Good for showing volume and spatial concentration

**Limitations:**
- Severe overplotting in dense areas (downtown, SoMa)
- Harder to see precise ZIP boundaries under points
- Requires sampling for performance (5k sample shown)
- Still single time snapshot - lacks temporal dimension

**Why rejected:** Overplotting obscures patterns in dense areas. Choropleth better for boundary-based regional questions.


In [ ]:
# Sample properties from 2023 for dot density
df_2023_sample = joined[joined['year'] == 2023].sample(n=5000, random_state=42)

# Base map with ZIP boundaries
base = alt.Chart(alt.Data(values=zip_2023_json['features'])).mark_geoshape(
    fill='none',
    stroke='lightgray',
    strokeWidth=0.5
).project(type='mercator')

# Dot density overlay
dots = alt.Chart(df_2023_sample).mark_circle(size=20, opacity=0.6).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color=alt.Color('total_assessed_value:Q',
                    scale=alt.Scale(scheme='viridis', domain=[p02, p98], clamp=True),
                    title='Property Value ($)'),
    tooltip=[
        alt.Tooltip('zip_code:N', title='ZIP'),
        alt.Tooltip('total_assessed_value:Q', format='$,.0f', title='Value')
    ]
).project(type='mercator')

(base + dots).properties(width=600, height=500, title='Property Value Distribution (2023, 5k sample)')


### Alternative 3: Proportional Symbols on Map

**Design:** Circles at ZIP centroids with size=count and color=median value.

**Advantages:**
- Shows both value (color) and volume (size) simultaneously
- Good for identifying dense vs sparse areas
- Clear visual hierarchy

**Limitations:**
- Symbol overlap in adjacent ZIPs (Marina/Pacific Heights)
- Harder to see exact ZIP boundaries under symbols
- Size and color compete for attention
- Still single time point - no temporal exploration

**Why rejected:** Choropleth better for boundary-based questions. Still lacks temporal component.


In [ ]:
# Create centroids
zip_centroids = zipcodes_gdf.copy()
zip_centroids['cx'] = zip_centroids.geometry.centroid.x
zip_centroids['cy'] = zip_centroids.geometry.centroid.y
zip_centroids = zip_centroids.merge(
    zip_yearly[zip_yearly['year'] == 2023], 
    on='zip_code'
)

# Base map (just boundaries)
base = alt.Chart(alt.Data(values=zip_2023_json['features'])).mark_geoshape(
    fill='lightgray', 
    stroke='white', 
    strokeWidth=0.5
).project(type='mercator')

# Proportional symbols
symbols = alt.Chart(zip_centroids).mark_circle(opacity=0.7).encode(
    longitude='cx:Q',
    latitude='cy:Q',
    size=alt.Size(
        'property_count:Q', 
        scale=alt.Scale(range=[100, 3000]), 
        title='# Properties'
    ),
    color=alt.Color(
        'median_value:Q', 
        scale=alt.Scale(scheme='viridis', domain=[p02, p98]), 
        title='Median Value'
    ),
    tooltip=[
        'zip_code', 
        alt.Tooltip('median_value:Q', format='$,.0f'), 
        alt.Tooltip('property_count:Q', format=',')
    ]
).project(type='mercator')

(base + symbols).properties(
    width=600, 
    height=500, 
    title='Property Value × Volume (2023)'
)

---

## Final Choice: Choropleth with Year Slider

**Why this is best:**

1. **Multi-scale temporal exploration:** Slider enables smooth navigation through all 9 years (2015-2023)
2. **Clear spatial boundaries:** Choropleth shows ZIP code regions distinctly  
3. **Interactive details:** Tooltips provide exact values on demand
4. **Consistent scale:** Fixed domain (2nd-98th percentile) enables valid year-to-year comparisons
5. **Efficient:** Single view encodes both spatial and temporal dimensions

**Encodings:**
- **Shape:** ZIP code polygon boundaries (geographic regions)
- **Color:** Median property value (sequential viridis scheme)
- **Slider:** Year selection (temporal navigation)
- **Tooltip:** Details on demand (ZIP, year, value, count)

**Justification:**
- Choropleth effective for aggregated regional data (MacEachren 1995)
- Sequential color schemes appropriate for magnitude (Brewer et al. 2003)
- Interactive controls superior to animation (Slocum et al. 2009)
- Viridis perceptually uniform and colorblind-friendly (Harrower & Brewer 2003)

**How it improves interpretability:**
1. **Addresses full question:** Shows both spatial distribution AND temporal evolution
2. **User control:** Slider provides precise year selection, better than forced animation
3. **Valid comparisons:** Fixed scale ensures colors mean same values across years
4. **Pattern discovery:** Quick year navigation reveals COVID impact (2020 dip), recovery (2021-2022), and appreciation patterns
5. **Multiple granularities:** Citywide patterns (color distribution) and individual ZIP details (tooltips)

In [ ]:
base_feats = json.loads(zipcodes_gdf[['zip_code', 'geometry']].to_json())['features']
val_map = {(str(r.zip_code), int(r.year)): (float(r.median_value), float(r.mean_value), int(r.property_count)) 
           for r in zip_yearly.itertuples(index=False)}

ymin, ymax = int(zip_yearly['year'].min()), int(zip_yearly['year'].max())

features_all = []
for f in base_feats:
    z = str(f['properties']['zip_code'])
    for y in range(ymin, ymax + 1):
        vals = val_map.get((z, y), (None, None, None))
        features_all.append({
            'type': 'Feature',
            'geometry': f['geometry'],
            'properties': {
                'zip_code': z,
                'year': int(y),
                'median_value': vals[0],
                'mean_value': vals[1],
                'property_count': vals[2]
            }
        })

year_param = alt.param('year', value=2023, bind=alt.binding_range(min=ymin, max=ymax, step=1, name='Year: '))

alt.Chart(alt.Data(values=features_all)).mark_geoshape(
    stroke='white', strokeWidth=1
).transform_filter(
    f'datum.properties.year == {year_param.name}'
).encode(
    color=alt.Color('properties.median_value:Q',
                    scale=alt.Scale(scheme='viridis', domain=[p02, p98], clamp=True),  # Added clamp=True
                    title='Median Value ($)'),
    tooltip=[
        alt.Tooltip('properties.zip_code:N', title='ZIP Code'),
        alt.Tooltip('properties.year:Q', title='Year'),
        alt.Tooltip('properties.median_value:Q', format='$,.0f', title='Median Value'),
        alt.Tooltip('properties.property_count:Q', format=',', title='# Properties')
    ]
).add_params(year_param).properties(
    width=700, height=600,
    title='SF Property Values by ZIP Code (2015-2023) - Outliers clamped to scale'
).project(type='mercator')